In [10]:

import numpy as np
import pandas as pd 
import decimal



In [11]:
def read_excel(path,sheet_name):
    df = pd.read_excel(io = path,sheet_name =sheet_name,  header = 0,index_col=None,)

   
    # name_column = df.columns.tolist()
    # for s in range(len(name_column)):
    # # del space start
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.lstrip()
    # # del space end
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.rstrip()
    return df

 
def to_json(df, nameFile):
    df.to_json(path_or_buf ='e:/DEV/FLUTTER/project/MY_GITHUB/dmitryitrofimov/json/thread/m/'+nameFile.lower()+'.json' , orient="records") 
    # df.to_json(path_or_buf ='../json/m/'+nameFile.lower()+'.json' , orient="records") 
    
def get_patch_json(file):
    return '../json/m/'+file+'.json' 

def get_patch_from(file):
    return '../excel/'+file+'.xlsx'

def change_types(df,columnType):
    for k, v in columnType.items(): 
        df[k] = df[k].astype(v)


def to_list_float(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [float(x) for x in df[nameCol].values[index].split(',')]

def to_list_String(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [str(x) for x in df[nameCol].values[index].split(',')]
            
            
def normalize_fraction(d):
    normalized = d.normalize()
    sign, digits, exponent = normalized.as_tuple()
    if exponent > 0:
        return decimal.Decimal((sign, digits + (0,) * exponent, 0))
    else:
        return normalized

def toStringWithoutZero(s):
    return s.rstrip('0').rstrip('.') if '.' in s else s        

# m_thread

In [12]:
file_excel = 'm_thread'

sheet_names = [
    'm_diam',
    'm_profile',
    'm_tolerance',
 
               ]

# ******************************
patchFrom = get_patch_from(file_excel)


df_base = read_excel(patchFrom, sheet_names[0])





In [13]:
df_m_diam = read_excel(patchFrom, sheet_names[0])
df_m_profile = read_excel(patchFrom, sheet_names[1])
df_m_tolerance = read_excel(patchFrom, sheet_names[2])

In [14]:
# column
diam = 'diam'
diamFormat = 'desc'
coarsePitch = 'coarsePitch'
finePitch = 'finePitch'
extraFinePitch = 'extraFinePitch'
rangeMain = 'rangeMain'
rangeSub = 'rangeSub'

# create df from df base

In [15]:
df_update = df_base[diam].unique()
df_update = pd.DataFrame( df_update, columns=[diam])

In [16]:
"""     required String desc,
    required String? coarsePitch,
    required List<String>? finePitch,
    required List<String>? extraFinePitch,
    required int? rangeMain,
    required int? rangeSub,
 """

'     required String desc,\n    required String? coarsePitch,\n    required List<String>? finePitch,\n    required List<String>? extraFinePitch,\n    required int? rangeMain,\n    required int? rangeSub,\n '

In [17]:
diam = 0
pitch = 0
for i, row in df_update.iterrows():
    diam = df_update.diam[i]
   
    
    
    df_update.at[i, 'desc']='M '+ toStringWithoutZero(str(diam))
    
    find_row =df_base.query("diam == @diam")
    if (find_row.count().sum())>0:
        range_main =  find_row.iloc[0]['range_main']
        range_sub =  find_row.iloc[0]['range_sub']
        
        df_update.at[i, rangeMain]=range_main
        df_update.at[i, rangeSub]=range_sub
    
    
    # coarse pitch
    find_row = df_base.query("diam == @diam and type_pitch == 1")
    if (find_row.count().sum())>0:
        pitch =  find_row.iloc[0]['pitch']
        df_update.at[i, coarsePitch]=toStringWithoutZero(str(pitch))
   
    # fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 2")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, finePitch]=str(pitch).replace('[','').replace(']','')  
    
    # extra fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 3")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, extraFinePitch]=str(pitch).replace('[','').replace(']','')  




# записываем диаметры

In [9]:
to_list_String(df_update,finePitch)
to_list_String(df_update,extraFinePitch)

df_update = df_update.drop(df_update.columns[[0]], axis=1)
to_json(df_update,'diam/diam')
    

In [193]:
coarsePitch = 'coarsePitch'

In [194]:
# df_pitch = pd.DataFrame(columns=[diamPitch])

for i, row in df_update.iterrows():
    df_pitch = pd.DataFrame()
   
    
    diamFormat = df_update.diamFormat[i]
    coarse = df_update.coarsePitch[i]
    fine = df_update.finePitch[i]
    extra = df_update.extraFinePitch[i]
   
   
    if not np.isnan(coarse):
        df_pitch.at[i, coarsePitch]=diamFormat +' x ' + str(coarse)
   
   
    #  отсеиваю все с nan
    if str(fine)!='nan':

        list2 = []
        for item in fine:
            s = str(item).replace('[','').replace(']','').replace(' ','')
            list2.append(diamFormat+ ' x ' +  toStringWithoutZero(s))
         
        
        df_pitch.at[i, finePitch]=str(list2)
        
    if str(extra)!='nan':

        list2 = []
        for item in extra:

            s = str(item).replace('[','').replace(']','').replace(' ','')
            list2.append(diamFormat+ ' x ' +  toStringWithoutZero(s))
         
        
        df_pitch.at[i, extraFinePitch]=str(list2)        
    
    
    # df_pitch[finePitch]= df_pitch[finePitch].astype('object')
    # to_list_float(df_pitch,finePitch)
    nameFile = diamFormat.replace(' ','')

    to_json(df_pitch, 'pitch/'+nameFile)
    
    # Read in the file
    with open(get_patch_json('pitch/'+nameFile), 'r') as file :
        filedata = file.read()

# Replace the target string
        filedata = filedata.replace('"[', '[').replace(']"', ']').replace("'", '"')

# Write the file out again
    with open(get_patch_json('pitch/'+nameFile), 'w') as file:
        file.write(filedata)
        
        
        
        
    
        

# write tolerance

In [195]:
df_tolerance = read_excel(patchFrom, sheet_names[2])
df_diam = read_excel(patchFrom, sheet_names[0])

def toleranceInfo(tolerance):
    return 'M '+ toStringWithoutZero(str(diam))+' x '+ toStringWithoutZero(str(pitch))+ ' - '+ tolerance
def uniq(x):
  return pd.unique(x).tolist()   

In [196]:

find_row =df_tolerance.query("diameter_min < @diam and diameter_max >= @diam and pitch ==@pitch")
    # print(find_row.shape())


In [197]:
diam = 0.0
pitch = 0.0


for i, row in df_diam.iterrows():
    diam = df_diam.diam[i]
    pitch = df_diam.pitch[i]


    if diam != 600:
        find_row =df_tolerance.query("diameter_min <= @diam and diameter_max > @diam and pitch ==@pitch")
    else :
        find_row =df_tolerance.query("diameter_min < @diam and diameter_max >= @diam and pitch ==@pitch")
        
    # print(find_row.shape())
    if (find_row.count().sum())==0: continue
# delete column if nan
    find_row2 = find_row.dropna(axis=1, how='all')
# deleted 1 2 3 column
    find_row2 = find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1)
# print(find_row2)
    column_headers = find_row2.columns.values.tolist()
    
    male = []
    female = []

    for item in column_headers:
        itemArray = str(item).split('_')
        if itemArray[3]=='m':
            male.append(toleranceInfo(itemArray[2]))
        else:
            female.append(toleranceInfo(itemArray[2].upper()))
    # оставляю уникальные
    male = uniq(male)    
    female = uniq(female)    
    
    df_male = pd.DataFrame(male, columns =['name'])
    df_female = pd.DataFrame(female, columns =['name'])
    print(str(diam) + " " + str(pitch))
    to_json(df_male,'tolerance/male/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))
    to_json(df_female,'tolerance/female/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))

1.0 0.25
1.0 0.2
1.1 0.25
1.1 0.2
1.2 0.25
1.2 0.2
1.4 0.2
1.6 0.35
1.6 0.2
1.8 0.35
1.8 0.2
2.0 0.4
2.0 0.25
2.2 0.45
2.2 0.25
2.5 0.45
2.5 0.35
3.0 0.5
3.0 0.35
3.5 0.5
3.5 0.6
3.5 0.35
4.0 0.35
4.0 0.25
4.0 0.7
4.0 0.5
4.5 0.35
4.5 0.25
4.5 0.75
4.5 0.5
5.0 0.35
5.0 0.25
5.0 0.8
5.0 0.5
5.5 0.35
5.5 0.25
5.5 0.5
6.0 0.35
6.0 0.25
6.0 1.0
6.0 0.75
6.0 0.5
6.5 0.75
6.5 0.5
6.5 0.35
6.5 0.25
7.0 0.35
7.0 0.25
7.0 1.0
7.0 0.75
7.0 0.5
7.5 0.75
7.5 0.5
7.5 0.35
7.5 0.25
8.0 0.35
8.0 0.25
8.0 1.25
8.0 1.0
8.0 0.75
8.0 0.5
8.5 1.0
8.5 0.75
8.5 0.5
9.0 0.35
9.0 1.25
9.0 1.0
9.0 0.75
9.0 0.5
9.5 1.0
9.5 0.75
9.5 0.5
9.5 0.35
10.0 0.35
10.0 1.5
10.0 1.25
10.0 1.0
10.0 0.75
10.0 0.5
10.5 1.0
10.5 0.75
10.5 0.5
11.0 0.35
11.0 1.5
11.0 1.0
11.0 0.75
11.0 0.5
11.5 1.0
11.5 0.75
11.5 0.5
12.0 0.35
12.0 1.75
12.0 1.5
12.0 1.25
12.0 1.0
12.0 0.75
12.0 0.5
12.5 1.5
12.5 1.0
12.5 0.75
12.5 0.5
13.0 1.5
13.0 1.0
13.0 0.75
13.0 0.5
13.5 1.5
13.5 1.0
13.5 0.75
13.5 0.5
14.0 2.0
14.0 1.5
14.0 1.25
14.0 1.

# get all info

In [198]:
diam = 10
pitch = 1
tolerance = '_4h_m'


In [199]:
find_row =df_tolerance.query("diameter_min <= @diam and diameter_max > @diam and pitch ==@pitch")
df_m_diam_find =df_m_diam.query("diam == @diam and pitch ==@pitch")

df_m_profile_find =df_m_profile.query("pitch == @pitch")

    # delete column if nan
find_row2 = find_row.dropna(axis=1, how='all')
   # deleted 1 2 3 column
find_row2 = find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,)
column_headers = find_row2.columns.values.tolist()
find_row3 = find_row2.filter(regex=tolerance)
find_row3 = find_row3.reset_index(drop=True)

find_row3.at[0, 'info']= 'M '+ toStringWithoutZero(str(diam))+' x ' + toStringWithoutZero(str(pitch))+' - ' +tolerance.split('_')[1] 
find_row3.at[0, 'diam']=toStringWithoutZero(str(diam))    
find_row3.at[0, 'pitch']=toStringWithoutZero(str(pitch))    
find_row3.at[0, 'tolerance']=tolerance.split('_')[1] 
# bolt 
isBolt =False
if tolerance.split('_')[2]=='m': 
    isBolt = True
find_row3.at[0, 'isBolt']= isBolt
#    
find_row3.at[0, 'pitch_diam_d2']= df_m_diam_find.pitch_diam_d2.iloc[0]
find_row3.at[0, 'minor_diam_d1']= df_m_diam_find.minor_diam_d1.iloc[0]
find_row3.at[0, 'minor_diam_d3']= df_m_diam_find.minor_diam_d3.iloc[0]
find_row3.at[0, 'type_pitch']= df_m_diam_find.type_pitch.iloc[0]
find_row3.at[0, 'range_main']= df_m_diam_find.range_main.iloc[0]
find_row3.at[0, 'range_sub']= df_m_diam_find.range_sub.iloc[0]

find_row3.at[0, 'h']= df_m_profile_find.h.iloc[0]
find_row3.at[0, 'h_5_8']= df_m_profile_find.h_5_8.iloc[0]
find_row3.at[0, 'h_3_8']= df_m_profile_find.h_3_8.iloc[0]
find_row3.at[0, 'h_4']= df_m_profile_find.h_4.iloc[0]
find_row3.at[0, 'h_8']= df_m_profile_find.h_8.iloc[0]




find_row3.columns = find_row3.columns.str.replace(tolerance, '')


In [200]:
for i, row in df_diam.iterrows():
    diam = df_diam.diam[i]
    pitch = df_diam.pitch[i]
    print(str(diam) + " " + str(pitch))


    if diam != 600:
        find_row =df_tolerance.query("diameter_min <= @diam and diameter_max > @diam and pitch ==@pitch")
    else :
        find_row =df_tolerance.query("diameter_min < @diam and diameter_max >= @diam and pitch ==@pitch")
        
    df_m_diam_find =df_m_diam.query("diam == @diam and pitch ==@pitch")
    df_m_profile_find =df_m_profile.query("pitch == @pitch")
    
    if (find_row.count().sum())==0: continue
    
    # delete column if nan
    find_row2 = find_row.dropna(axis=1, how='all')
   # deleted 1 2 3 column
    find_row2 = find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,)
    
    
     
    male = []
    female = []
    column_headers = find_row2.columns.values.tolist()

    for item in column_headers:
        itemArray = str(item).split('_')
        if itemArray[3]=='m':
            male.append('_'+itemArray[2]+'_m')
        else:
            female.append('_'+itemArray[2]+'_f')
            
       # оставляю уникальные
    male = uniq(male)    
    female = uniq(female)    
    

   
    for tolerance in male:
   
        find_row3 = find_row2.filter(regex=tolerance)
        find_row3 = find_row3.reset_index(drop=True)
        

        find_row3.at[0, 'info']= 'M '+ toStringWithoutZero(str(diam))+' x ' + toStringWithoutZero(str(pitch))+' - ' +tolerance.split('_')[1] 
        find_row3.at[0, 'diam']=toStringWithoutZero(str(diam))    
        find_row3.at[0, 'pitch']=toStringWithoutZero(str(pitch))    
        find_row3.at[0, 'tolerance']=tolerance.split('_')[1] 
        # bolt 
        isBolt =False
        if tolerance.split('_')[2]=='m': 
            isBolt = True
        find_row3.at[0, 'isBolt']= isBolt
        #  
        find_row3.at[0, 'pitch_diam_d2']= df_m_diam_find.pitch_diam_d2.iloc[0]
        find_row3.at[0, 'minor_diam_d1']= df_m_diam_find.minor_diam_d1.iloc[0]
        find_row3.at[0, 'minor_diam_d3']= df_m_diam_find.minor_diam_d3.iloc[0]
        find_row3.at[0, 'type_pitch']= df_m_diam_find.type_pitch.iloc[0]
        find_row3.at[0, 'range_main']= df_m_diam_find.range_main.iloc[0]
        find_row3.at[0, 'range_sub']= df_m_diam_find.range_sub.iloc[0]

        find_row3.at[0, 'h']= df_m_profile_find.h.iloc[0]
        find_row3.at[0, 'h_5_8']= df_m_profile_find.h_5_8.iloc[0]
        find_row3.at[0, 'h_3_8']= df_m_profile_find.h_3_8.iloc[0]
        find_row3.at[0, 'h_4']= df_m_profile_find.h_4.iloc[0]
        find_row3.at[0, 'h_8']= df_m_profile_find.h_8.iloc[0]
   
        find_row3.columns = find_row3.columns.str.replace(tolerance, '')
        to_json(find_row3,'info/male/m'+toStringWithoutZero(str(diam))+'x'+toStringWithoutZero(str(pitch))+'-'+tolerance.split('_')[1])

    for tolerance in female:
   
        find_row3 = find_row2.filter(regex=tolerance)
        find_row3 = find_row3.reset_index(drop=True)
        

        find_row3.at[0, 'info']= 'M '+ toStringWithoutZero(str(diam))+' x ' + toStringWithoutZero(str(pitch))+' - ' +tolerance.split('_')[1].upper()  
        find_row3.at[0, 'diam']=toStringWithoutZero(str(diam))    
        find_row3.at[0, 'pitch']=toStringWithoutZero(str(pitch))    
        find_row3.at[0, 'tolerance']=tolerance.split('_')[1].upper() 
        # bolt 
        isBolt =False
        if tolerance.split('_')[2]=='m': 
            isBolt = True
        find_row3.at[0, 'isBolt']= isBolt
        #  
        find_row3.at[0, 'pitch_diam_d2']= df_m_diam_find.pitch_diam_d2.iloc[0]
        find_row3.at[0, 'minor_diam_d1']= df_m_diam_find.minor_diam_d1.iloc[0]
        find_row3.at[0, 'minor_diam_d3']= df_m_diam_find.minor_diam_d3.iloc[0]
        find_row3.at[0, 'type_pitch']= df_m_diam_find.type_pitch.iloc[0]
        find_row3.at[0, 'range_main']= df_m_diam_find.range_main.iloc[0]
        find_row3.at[0, 'range_sub']= df_m_diam_find.range_sub.iloc[0]

        find_row3.at[0, 'h']= df_m_profile_find.h.iloc[0]
        find_row3.at[0, 'h_5_8']= df_m_profile_find.h_5_8.iloc[0]
        find_row3.at[0, 'h_3_8']= df_m_profile_find.h_3_8.iloc[0]
        find_row3.at[0, 'h_4']= df_m_profile_find.h_4.iloc[0]
        find_row3.at[0, 'h_8']= df_m_profile_find.h_8.iloc[0]
   
        find_row3.columns = find_row3.columns.str.replace(tolerance, '')
        to_json(find_row3,'info/female/m'+toStringWithoutZero(str(diam))+'x'+toStringWithoutZero(str(pitch))+'-'+tolerance.split('_')[1])
# print(female)
# print(find_row3)



0.25 0.075
0.3 0.08
0.35 0.09
0.4 0.1
0.45 0.1
0.5 0.125
0.55 0.125
0.6 0.15
0.7 0.175
0.8 0.2
0.9 0.225
1.0 0.25
1.0 0.2
1.1 0.25
1.1 0.2
1.2 0.25
1.2 0.2
1.4 0.3
1.4 0.2
1.6 0.35
1.6 0.2
1.8 0.35
1.8 0.2
2.0 0.4
2.0 0.25
2.2 0.45
2.2 0.25
2.5 0.45
2.5 0.35
3.0 0.5
3.0 0.35
3.5 0.5
3.5 0.6
3.5 0.35
4.0 0.35
4.0 0.25
4.0 0.7
4.0 0.5
4.5 0.35
4.5 0.25
4.5 0.75
4.5 0.5
5.0 0.35
5.0 0.25
5.0 0.8
5.0 0.5
5.5 0.35
5.5 0.25
5.5 0.5
6.0 0.35
6.0 0.25
6.0 1.0
6.0 0.75
6.0 0.5
6.5 0.75
6.5 0.5
6.5 0.35
6.5 0.25
7.0 0.35
7.0 0.25
7.0 1.0
7.0 0.75
7.0 0.5
7.5 0.75
7.5 0.5
7.5 0.35
7.5 0.25
8.0 0.35
8.0 0.25
8.0 1.25
8.0 1.0
8.0 0.75
8.0 0.5
8.5 1.0
8.5 0.75
8.5 0.5
9.0 0.35
9.0 1.25
9.0 1.0
9.0 0.75
9.0 0.5
9.5 1.0
9.5 0.75
9.5 0.5
9.5 0.35
10.0 0.35
10.0 1.5
10.0 1.25
10.0 1.0
10.0 0.75
10.0 0.5
10.5 1.0
10.5 0.75
10.5 0.5
11.0 0.35
11.0 1.5
11.0 1.0
11.0 0.75
11.0 0.5
11.5 1.0
11.5 0.75
11.5 0.5
12.0 0.35
12.0 1.75
12.0 1.5
12.0 1.25
12.0 1.0
12.0 0.75
12.0 0.5
12.5 1.5
12.5 1.0
12.5 0.75
12.5 